<a href="https://colab.research.google.com/github/Malik-Raheel/Trading-Strategies/blob/main/Alpha_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Testing Virat AGents

In [2]:
pip install ccxt pandas numpy ta scikit-learn matplotlib seaborn requests


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.6/288.6 kB 14.9 MB/s eta 0:00:00
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=25da3e12be5317fd15b5aff96d577f799e62e8318262d000a23b947536314a85
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.11.11
    Uninstalling aiohttp-3.11.11:
      Successfully uninstalled aiohttp-3.11.11


In [3]:
import ccxt
import pandas as pd
import numpy as np
import time

def fetch_data(exchange, symbol, timeframe='1h', limit=100):
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe=timeframe, limit=limit)
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    return df

def calculate_indicators(df):
    # Calculate OBV
    df['obv'] = (np.sign(df['close'].diff()) * df['volume']).fillna(0).cumsum()

    # Calculate ATR
    df['tr'] = np.maximum(df['high'] - df['low'],
                          np.maximum(abs(df['high'] - df['close'].shift()),
                                     abs(df['low'] - df['close'].shift())))
    df['atr'] = df['tr'].rolling(window=14).mean()

    # Calculate ADX
    df['plus_dm'] = np.where((df['high'] - df['high'].shift()) > (df['low'].shift() - df['low']),
                             np.maximum(df['high'] - df['high'].shift(), 0), 0)
    df['minus_dm'] = np.where((df['low'].shift() - df['low']) > (df['high'] - df['high'].shift()),
                              np.maximum(df['low'].shift() - df['low'], 0), 0)
    df['tr14'] = df['tr'].rolling(window=14).sum()
    df['plus_di'] = 100 * (df['plus_dm'].rolling(window=14).sum() / df['tr14'])
    df['minus_di'] = 100 * (df['minus_dm'].rolling(window=14).sum() / df['tr14'])
    df['dx'] = 100 * abs(df['plus_di'] - df['minus_di']) / (df['plus_di'] + df['minus_di'])
    df['adx'] = df['dx'].rolling(window=14).mean()

    # Calculate Z-Score
    df['mean'] = df['close'].rolling(window=20).mean()
    df['std'] = df['close'].rolling(window=20).std()
    df['z_score'] = (df['close'] - df['mean']) / df['std']

    return df

def analyze_asset(symbol, df):
    last_row = df.iloc[-1]

    # Interpret ADX
    adx = last_row['adx']
    if adx < 20:
        adx_interpretation = "Weak trend"
    elif 20 <= adx < 40:
        adx_interpretation = "Moderate trend"
    else:
        adx_interpretation = "Strong trend"

    # Interpret Z-Score
    z_score = last_row['z_score']
    if z_score < -1:
        z_score_interpretation = "Oversold (price significantly below mean)"
    elif z_score > 1:
        z_score_interpretation = "Overbought (price significantly above mean)"
    else:
        z_score_interpretation = "Neutral (price near mean)"

    return {
        "symbol": symbol,
        "obv": last_row['obv'],
        "atr": last_row['atr'],
        "adx": adx,
        "adx_interpretation": adx_interpretation,
        "z_score": z_score,
        "z_score_interpretation": z_score_interpretation
    }

def main():
    exchange = ccxt.kucoin()
    assets = ['BTC/USDT', 'NEAR/USDT', 'HNT/USDT', 'ETH/USDT', 'XAVA/USDT',
              'STG/USDT', 'SOL/USDT', 'ADA/USDT', 'XRP/USDT', 'LINK/USDT']

    results = []

    for asset in assets:
        try:
            df = fetch_data(exchange, asset)
            df = calculate_indicators(df)
            analysis = analyze_asset(asset, df)
            results.append(analysis)
        except Exception as e:
            print(f"Error processing {asset}: {e}")

    # Display Results
    print("\nAsset Analysis Results:")
    for result in results:
        print(f"\nSymbol: {result['symbol']}\n"
              f"OBV: {result['obv']:.2f}\n"
              f"ATR: {result['atr']:.4f} (Volatility Measure)\n"
              f"ADX: {result['adx']:.2f} ({result['adx_interpretation']})\n"
              f"Z-Score: {result['z_score']:.2f} ({result['z_score_interpretation']})")

if __name__ == "__main__":
    main()



Asset Analysis Results:

Symbol: BTC/USDT
OBV: -757.22
ATR: 497.9714 (Volatility Measure)
ADX: 21.03 (Moderate trend)
Z-Score: -1.05 (Oversold (price significantly below mean))

Symbol: NEAR/USDT
OBV: -128649.08
ATR: 0.0610 (Volatility Measure)
ADX: 43.88 (Strong trend)
Z-Score: -1.13 (Oversold (price significantly below mean))

Symbol: HNT/USDT
OBV: 61251.16
ATR: 0.0624 (Volatility Measure)
ADX: 69.70 (Strong trend)
Z-Score: -1.05 (Oversold (price significantly below mean))

Symbol: ETH/USDT
OBV: 15834.33
ATR: 23.5521 (Volatility Measure)
ADX: 56.04 (Strong trend)
Z-Score: -0.92 (Neutral (price near mean))

Symbol: XAVA/USDT
OBV: 47894.30
ATR: 0.0064 (Volatility Measure)
ADX: 35.10 (Moderate trend)
Z-Score: -0.78 (Neutral (price near mean))

Symbol: STG/USDT
OBV: 311586.80
ATR: 0.0036 (Volatility Measure)
ADX: 15.40 (Weak trend)
Z-Score: -1.05 (Oversold (price significantly below mean))

Symbol: SOL/USDT
OBV: 190886.04
ATR: 3.2864 (Volatility Measure)
ADX: 65.13 (Strong trend)
Z-Scor

# **Average Directional Index (ADX)**
Purpose: Measures trend strength.
Usage:
A high ADX (e.g., above 25) suggests a strong trend (either up or down).
A low ADX suggests a weak or sideways market.

# **Z-Score**
Purpose: Measures how far the current price deviates from its rolling mean (mean reversion indicator).
Usage:
A low Z-Score (e.g., below -1) indicates the price is significantly below its mean, potentially offering a bullish opportunity for mean reversion.